# Avance 1 — Análisis Exploratorio de Clientes
## Proyecto: Conociendo al Cliente 360°
**Autor:** Cristian Fernando García Cadena  
**Fecha:** 23 de Octubre de 2025  

---

### Objetivos
- Cargar y explorar la base de datos de clientes.  
- Evaluar calidad, duplicados y errores.  
- Filtrar y analizar por ciudad.  
- Crear tablas de resumen (género, preferencias, consumo, etc.).  

---

> Este notebook forma parte del pipeline analítico inicial del proyecto, cuyo propósito es evaluar y preparar los datos para futuras integraciones con fuentes externas.


In [162]:
# ============================================
# Configuración inicial
# ============================================

# Librerías básicas
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Cargar variables de entorno (.env)
from dotenv import load_dotenv
load_dotenv()

# Comprobar entorno y versión
print("Entorno activo:", os.getcwd())
print("Versión de pandas:", pd.__version__)


Entorno activo: c:\Users\Diego Garcia\Documents\Cristian\DS-SoyHenry\Modulo 1\Código\notebooks
Versión de pandas: 2.3.3


In [163]:
# ============================================
# Carga y exploración de la base de datos
# ============================================

from pathlib import Path

# Ruta del archivo principal (ajusta nombre si difiere)
path_clientes = DATA_RAW / "base_datos_restaurantes_USA_v2.csv"

# Leer datos
df = pd.read_csv(path_clientes, encoding="latin-1")

# Vista preliminar
df.head(5)



,id_persona,nombre,apellido,edad,genero,ciudad_residencia,estrato_socioeconomico,frecuencia_visita,promedio_gasto_comida,ocio,consume_licor,preferencias_alimenticias,membresia_premium,telefono_contacto,correo_electronico,tipo_de_pago_mas_usado,ingresos_mensuales
0,2550327378,Jackson,Gomez,31.0,Masculino,Miami,Alto,6,67.51,SÃ­,No,Vegetariano,SÃ­,(830)220-1926,NaN,Efectivo,6425
1,9446112038,Samantha,Soto,40.0,Femenino,Denver,Medio,2,44.92,SÃ­,SÃ­,Mariscos,No,881-476-1426,NaN,Efectivo,2374
2,3098363243,Terry,Adams,62.0,Femenino,Denver,Bajo,2,9.24,SÃ­,SÃ­,Vegetariano,No,NaN,diana74@example.net,Efectivo,1110
3,4013002847,James,Shannon,41.0,Masculino,Boston,Alto,5,30.74,SÃ­,SÃ­,Carnes,SÃ­,NaN,scottfrey@example.com,Tarjeta,6931
4,7372911048,Susan,Jones,49.0,Femenino,San Diego,Bajo,0,0.00,No,No,Carnes,No,243.248.8919,glassgary@example.org,Tarjeta,1350


Objetivo: normalizar respuestas binarias para evitar categorías “fantasma” como sã, sÃ­, si vs sí.
Justificación: si no lo hacemos al inicio, las tabulaciones y filtros producirán conteos erróneos y dificultarán cualquier análisis (ej. métricas por “sí/no”). Lo hacemos antes de la sección de tabulación para que todo el EDA use valores consistentes.

In [164]:
# === Normalización fuerte de respuestas binarias ===

import re
from unicodedata import normalize

# Columnas objetivo (ajústalas si agregas/quitas binarias)
BINARIAS = ["consume_licor", "ocio", "membresia_premium"]

# Caracteres invisibles/problemáticos frecuentes en CSVs:
#  \u00AD = soft hyphen, \u200B-\u200F = zero-widths, \uFEFF = BOM, \u00A0 = NBSP (espacio no separable)
_INVISIBLES_RE = re.compile(r"[\u00AD\u200B-\u200F\uFEFF\u00A0]")

def normalize_yes_no(x):
    """
    Normaliza 'sí'/'no' corrigiendo:
    - mojibake (sÃ­, sã, sí́, sì),
    - caracteres invisibles (soft hyphen, zero-widths, BOM, NBSP),
    - espacios y mayúsculas/minúsculas,
    - variantes 'si'/'sí' y 'no'/'nó'.
    Mantiene NaN sin tocar.
    """
    if pd.isna(x):
        return x

    # a texto
    t = str(x)

    # 1) Unicode canónico (evita acentos compuestos distintos)
    t = normalize("NFC", t)

    # 2) Quitar invisibles (soft hyphen, zero-width, BOM, NBSP)
    t = _INVISIBLES_RE.sub("", t)

    # 3) Normalizar espacios y casing
    t = t.strip().lower()

    # 4) Reparar mojibake y variantes comunes de 'sí'
    # (si aparece otra variante rara, agrégala aquí)
    repl = {
        "sÃ­": "sí",
        "sã": "sí",
        "sí́": "sí",  # acento combinado
        "sì": "sí",
        "sÍ": "sí",
    }
    t = repl.get(t, t)

    # 5) Unificación final a dos valores canónicos
    if t in {"si", "sí"}:
        return "sí"
    if t in {"no", "nó"}:
        return "no"

    # Si aparece otro valor (p.ej. 'desconocido'), lo dejamos para detectarlo luego
    return t

# Aplicar solo a las columnas especificadas si existen
for col in BINARIAS:
    if col in df.columns:
        df[col] = df[col].map(normalize_yes_no)

# (Opcional) Verificación rápida
for col in BINARIAS:
    if col in df.columns:
        print(f"\n{col}:")
        print(df[col].value_counts(dropna=False))
        print("unique:", df[col].unique())



consume_licor:
consume_licor
sí    18483
no    11517
Name: count, dtype: int64
unique: ['no' 'sí']

ocio:
ocio
no    15094
sí    14906
Name: count, dtype: int64
unique: ['sí' 'no']

membresia_premium:
membresia_premium
no    17155
sí    12845
Name: count, dtype: int64
unique: ['sí' 'no']


In [165]:
# ============================================
# Inspección inicial del contenido
# ============================================

display(df.head(3))

print("\nINFO df")
df.info()

print("\nColumnas df: ", list(df.columns))

,id_persona,nombre,apellido,edad,genero,ciudad_residencia,estrato_socioeconomico,frecuencia_visita,promedio_gasto_comida,ocio,consume_licor,preferencias_alimenticias,membresia_premium,telefono_contacto,correo_electronico,tipo_de_pago_mas_usado,ingresos_mensuales
0,2550327378,Jackson,Gomez,31.0,Masculino,Miami,Alto,6,67.51,sí,no,Vegetariano,sí,(830)220-1926,NaN,Efectivo,6425
1,9446112038,Samantha,Soto,40.0,Femenino,Denver,Medio,2,44.92,sí,sí,Mariscos,no,881-476-1426,NaN,Efectivo,2374
2,3098363243,Terry,Adams,62.0,Femenino,Denver,Bajo,2,9.24,sí,sí,Vegetariano,no,NaN,diana74@example.net,Efectivo,1110



INFO df
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id_persona                 30000 non-null  int64  
 1   nombre                     30000 non-null  object 
 2   apellido                   30000 non-null  object 
 3   edad                       29899 non-null  float64
 4   genero                     30000 non-null  object 
 5   ciudad_residencia          30000 non-null  object 
 6   estrato_socioeconomico     30000 non-null  object 
 7   frecuencia_visita          30000 non-null  int64  
 8   promedio_gasto_comida      29855 non-null  float64
 9   ocio                       30000 non-null  object 
 10  consume_licor              30000 non-null  object 
 11  preferencias_alimenticias  28597 non-null  object 
 12  membresia_premium          30000 non-null  object 
 13  telefono_contacto          14834 non-

In [166]:
# ============================================
# Vderificaciónd de valores nulos y duplicados
# ============================================

# Verificar nulos y duplicados
print("Duplicados:", df.duplicated().sum())
print("\nValores nulos por columna (%):")
print((df.isna().sum() / len(df) * 100).round(2))


Duplicados: 0

Valores nulos por columna (%):
id_persona                    0.00
nombre                        0.00
apellido                      0.00
edad                          0.34
genero                        0.00
ciudad_residencia             0.00
estrato_socioeconomico        0.00
frecuencia_visita             0.00
promedio_gasto_comida         0.48
ocio                          0.00
consume_licor                 0.00
preferencias_alimenticias     4.68
membresia_premium             0.00
telefono_contacto            50.55
correo_electronico           50.24
tipo_de_pago_mas_usado        0.00
ingresos_mensuales            0.00
dtype: float64


In [167]:
# ============================================
# Función para tabular distribuciones
# ============================================

def dist_table(series, top=None, normalize_text=True):
    """
    Devuelve una tabla con conteos y porcentajes de una serie categórica.
    - normalize_text=True: sólo para el cálculo, estandariza a minúsculas y sin espacios
      para evitar que 'Femenino', 'femenino ' y 'FEMENINO' cuenten como categorías distintas.
    - top: si se indica, muestra solo las 'top' categorías más frecuentes.
    NO modifica el DataFrame original.
    """
    s = series.copy()

    # Para no alterar df: normalizamos SOLO para contar (mantenemos NaN)
    if normalize_text and s.dtype == "object":
        s = s.astype("string")
        s = s.str.strip().str.lower()

    # Conteos y porcentajes (incluimos NaN explícitamente)
    counts = s.value_counts(dropna=False)
    perc = s.value_counts(normalize=True, dropna=False).mul(100).round(2)

    out = pd.DataFrame({"count": counts, "porcentaje_%": perc})

    # Si se pide top-n
    if top is not None:
        out = out.head(top)

    return out


In [168]:
# ============================================
# Tabulaciones para verificar distribuciones
# ============================================

print("== Distribución: genero ==")
display(dist_table(df["genero"], top=None))

print("\n== Distribución: preferencias_alimenticias ==")
display(dist_table(df["preferencias_alimenticias"], top=10))

print("\n== Distribución: consume_licor ==")
display(dist_table(df["consume_licor"], top=None))

print("== Distribución: estrato_socioeconomico ==")
display(dist_table(df["estrato_socioeconomico"], top=None))

print("\n== Distribución: ocio ==")
display(dist_table(df["ocio"], top=10))

print("\n== Distribución: membresia_premium ==")
display(dist_table(df["membresia_premium"], top=None))

print("\n== Distribución: tipo_de_pago_mas_usado ==")
display(dist_table(df["tipo_de_pago_mas_usado"], top=10))

== Distribución: genero ==


,count,porcentaje_%
genero,,
femenino,15044,50.15
masculino,14956,49.85



== Distribución: preferencias_alimenticias ==


,count,porcentaje_%
preferencias_alimenticias,,
carnes,7916,26.39
vegetariano,6580,21.93
mariscos,5212,17.37
vegano,3267,10.89
pescado,2983,9.94
otro,2639,8.8
<NA>,1403,4.68



== Distribución: consume_licor ==


...

,count,porcentaje_%
ocio,,
no,15094,50.31
sí,14906,49.69



== Distribución: membresia_premium ==


,count,porcentaje_%
membresia_premium,,
no,17155,57.18
sí,12845,42.82



== Distribución: tipo_de_pago_mas_usado ==


,count,porcentaje_%
tipo_de_pago_mas_usado,,
efectivo,11813,39.38
tarjeta,9954,33.18
app,7678,25.59
criptomoneda,555,1.85


## Análisis de variables (tabulación básica, sin filtro por ciudad)

**Contexto general**
- `n = 30.000` registros, `17` columnas.
- Nulos destacables: `telefono_contacto` (50,55%), `correo_electronico` (50,24%), `preferencias_alimenticias` (4,68%), `edad` (0,34%), `promedio_gasto_comida` (0,48%).
- No hay duplicados de filas.

### 1) género
- Distribución equilibrada: **femenino 50,15%**, **masculino 49,85%**.
- Sin nulos.
- **Conclusión:** variable lista para análisis comparativo por sexo. No requiere limpieza adicional.

### 2) preferencias_alimenticias
- Distribución principal: carnes (26,39%), vegetariano (21,93%), mariscos (17,37%), vegano (10,89%), pescado (9,94%) y “otro” (8,80%).
- Nulos: **4,68%** → tamaño moderado.
- **Conclusión:** categorías parecen coherentes y mutuamente excluyentes. Los nulos pueden tratarse más adelante (etiqueta “Sin dato”) si el caso de uso lo requiere.

### 3) consume_licor
- Binaria: **sí ≈ 61,6%**, **no ≈ 38,4%**.
- Se detectó un problema de codificación (p.ej. `sã`) que fue normalizado tempranamente a **“sí”**.
- **Conclusión:** tras normalizar, lista para análisis (comparativos con gasto/visita).

### 4) estrato_socioeconomico
- Distribución plausible: **medio (31,1%)**, **alto (30,1%)**, **bajo (20,5%)**, **muy alto (18,3%)**.
- Sin nulos.
- **Conclusión:** variable categórica **ordenable** (bajo < medio < alto < muy alto). En caso dado de una modelación futura, convendría mapearla a ordinal.

### 5) ocio
- Binaria ~50/50 (sí/no). Se observó el mismo problema de codificación que en `consume_licor`, ya corregido.
- **Conclusión:** tras normalización, lista para segmentaciones (p.ej., ocio vs. gasto).

### 6) membresia_premium
- Binaria con **no (57,2%)** y **sí (42,8%)** (tras normalización).
- **Conclusión:** variable clave para análisis de valor del cliente (gasto, frecuencia).

### 7) tipo_de_pago_mas_usado
- Distribución lógica: **efectivo (39,4%)**, **tarjeta (33,2%)**, **app (25,6%)**, **criptomoneda (1,85%)**.
- Sin nulos aparentes.
- **Conclusión:** lista para análisis; considerar agrupar “app” vs métodos tradicionales en comparativas.

### Observaciones transversales
- Las variables binarias requerían **normalización de “sí/no”** por errores de codificación. Ya se corrigió.
- Las categorías parecen consistentes; no se observan mezclas obvias (ej. “Tarjeta” vs “tarjeta de crédito”).
- En una etapa posterior del curso, y evaluando conocimientos posteriores, conviene:
  - Codificar `estrato_socioeconomico` como **ordinal**.
  - Evaluar imputación de nulos en `preferencias_alimenticias` (4,68%) si bloquea análisis.
  - Decidir el uso de `telefono_contacto`/`correo_electronico`: alto porcentaje de nulos sugiere excluirlos de análisis cuantitativo.


Objetivo: elegir una ciudad con tamaño de muestra suficiente y relevancia para el análisis posterior.

Criterios pensados inicialmente:

Top ciudades por número de clientes (asegura estabilidad estadística).

Alternativamente, ciudades con presencia balanceada de membresia_premium o alta variabilidad en promedio_gasto_comida (comparar comportamientos).

In [169]:
# === Exploración de ciudades para definir el filtro ===

# 1) Top ciudades por cantidad absoluta
top_cities = (df["ciudad_residencia"]
              .astype("string").str.strip()
              .value_counts()
              .rename_axis("ciudad")
              .reset_index(name="count"))
top_cities["porcentaje_%"] = (top_cities["count"] / len(df) * 100).round(2)
display(top_cities)

# 2) Métricas rápidas por ciudad para criterio adicional
#    - tasa de premium
#    - gasto promedio
city_metrics = (df.groupby(df["ciudad_residencia"].astype("string").str.strip(), dropna=False)
                  .agg(
                      n_clientes=("id_persona", "size"),
                      tasa_premium=("membresia_premium", lambda s: (s == "sí").mean()*100),
                      gasto_prom=("promedio_gasto_comida", "mean")
                  )
                  .sort_values("n_clientes", ascending=False)
                  .head(20)
                  .round({"tasa_premium": 2, "gasto_prom": 2}))
display(city_metrics)


,ciudad,count,porcentaje_%
0,Chicago,5384,17.95
1,NYC,4769,15.9
2,Miami,3186,10.62
3,San Diego,3075,10.25
4,Dallas,2602,8.67
5,Boston,2547,8.49
6,Denver,2523,8.41
7,Houston,2212,7.37
8,Seattle,2191,7.3
9,Phoenix,1511,5.04


,n_clientes,tasa_premium,gasto_prom
ciudad_residencia,,,
Chicago,5384,41.14,25.53
NYC,4769,37.22,37.12
Miami,3186,53.17,39.82
San Diego,3075,48.85,34.75
Dallas,2602,38.12,28.70
Boston,2547,44.72,29.49
Denver,2523,42.33,34.26
Houston,2212,36.53,26.02
Seattle,2191,46.69,36.86


# Análisis por Ciudad

---

## Distribución General de Clientes

Los datos muestran que el número de clientes no se reparte de manera uniforme entre las ciudades.  
La mayor concentración se da en las siguientes:

| Ciudad     | Participación (%) | Clientes |
|-------------|:----------------:|:--------:|
| **Chicago** | 17.95 | 5,384 |
| **NYC**     | 15.90 | 4,769 |
| **Miami**   | 10.62 | 3,186 |
| **San Diego** | 10.25 | 3,075 |
| **Dallas**  | 8.67  | 2,602 |
| **Boston**  | 8.49  | 2,547 |

Estas seis ciudades concentran más del **70% del total de registros**, lo que indica que el análisis posterior debería centrarse en una de ellas para representar adecuadamente la población de clientes.

---

## Gasto Promedio por Comida

El gasto promedio (en miles de USD) varía de manera apreciable entre las ciudades:

| Ciudad     | Gasto Promedio |
|-------------|:--------------:|
| **Miami**   | 39.82 |
| **NYC**     | 37.12 |
| **Seattle** | 36.86 |
| **Phoenix** | 36.54 |
| **San Diego** | 34.75 |
| **Boston**  | 29.49 |
| **Dallas**  | 28.70 |
| **Houston** | 26.02 |
| **Chicago** | 25.53 |

**Miami, NYC y Seattle** muestran los **mayores promedios de gasto**, lo cual podría reflejar mayor poder adquisitivo o un perfil de cliente más activo en consumo.  
Por otro lado, **Chicago**, a pesar de tener la mayor cantidad de clientes, presenta **el gasto promedio más bajo**; esto sugiere una base amplia pero menos intensiva en consumo.

---

## Tasa de Membresía Premium

Tras la corrección de datos, se observa que las tasas de membresía premium presentan diferencias importantes entre ciudades.

En general:
- **Miami** y **NYC** tienden a mostrar **altas tasas premium**, coherentes con sus mayores niveles de gasto promedio.  
- **Chicago** combina **alto volumen de clientes** pero **baja proporción de miembros premium**, lo cual indica un mercado masivo pero con espacio para estrategias de fidelización.  
- **Seattle** y **San Diego** muestran una relación equilibrada entre gasto y membresía, posiblemente segmentos maduros y con buen comportamiento de compra.

---

## Conclusiones del Análisis

1. **Volumen vs. Valor**  
   - Ciudades como **Chicago** aportan volumen de clientes, pero no necesariamente valor individual.  
   - **Miami** y **NYC** destacan por su gasto y fidelización.

2. **Segmentos de Oportunidad**  
   - **Chicago**: oportunidad para campañas de conversión a membresía premium.  
   - **Boston/Dallas**: gasto medio, posibles candidatos a estrategias de retención.

3. **Segmentos de Rentabilidad**  
   - **Miami** y **NYC** son los mercados más rentables por gasto promedio y tasa premium.  
   - Ideal para análisis inicial del comportamiento de clientes de alto valor.

---

## Propuesta de Filtrado

Para continuar con el análisis exploratorio y la limpieza detallada, se recomienda filtrar por **una sola ciudad** que combine:

- Tamaño de muestra representativo (≥10% del total de clientes).  
- Alta calidad de datos.  
- Patrón de consumo significativo.

### Ciudad Sugerida: **Miami**

**Justificación:**
- Tiene un volumen de datos suficiente (**10.6%** del total).  
- Exhibe **el gasto promedio más alto**.  
- Presenta **una de las tasas premium más elevadas**, reflejando clientes más fidelizados y de mayor poder adquisitivo.  
- Permite construir un modelo de cliente más orientado a **rentabilidad** que a **volumen**.

---

In [170]:
# ============================================
# Filtrado del DataFrame por ciudad: Miami
# ============================================

# Normalizamos el texto en 'ciudad_residencia' antes de comparar
# (para evitar errores por espacios o mayúsculas)
df["ciudad_residencia"] = df["ciudad_residencia"].str.strip().str.lower()

# Creamos un nuevo subconjunto con solo registros de la ciudad seleccionada
df_miami = df[df["ciudad_residencia"] == "miami"].copy()

# ============================================
# Verificación de integridad del filtrado
# ============================================

# Confirmamos dimensiones del nuevo subconjunto
print("Dimensiones del subconjunto (Miami):", df_miami.shape)

# Mostramos conteos básicos para validar coherencia
print("\nDistribución de membresía premium en Miami:")
print(df_miami["membresia_premium"].value_counts(normalize=True).mul(100).round(2))

print("\nPromedio de gasto en comida:")
print(df_miami["promedio_gasto_comida"].mean().round(2))

# Verificamos nulos por columna (solo vista general)
print("\nValores nulos por columna (en %):")
print(df_miami.isna().mean().mul(100).round(2))


Dimensiones del subconjunto (Miami): (3186, 17)

Distribución de membresía premium en Miami:
membresia_premium
sí    53.17
no    46.83
Name: proportion, dtype: float64

Promedio de gasto en comida:
39.82

Valores nulos por columna (en %):
id_persona                    0.00
nombre                        0.00
apellido                      0.00
edad                          0.25
genero                        0.00
ciudad_residencia             0.00
estrato_socioeconomico        0.00
frecuencia_visita             0.00
promedio_gasto_comida         0.50
ocio                          0.00
consume_licor                 0.00
preferencias_alimenticias     4.58
membresia_premium             0.00
telefono_contacto            50.09
correo_electronico           49.87
tipo_de_pago_mas_usado        0.00
ingresos_mensuales            0.00
dtype: float64


## Justificación del Filtrado por Ciudad: Miami

La elección de **Miami** como ciudad de referencia responde a criterios tanto analíticos como estratégicos dentro del proceso de exploración de datos.

### Criterios Cuantitativos

1. **Representatividad del Muestra**  
   Miami concentra el **10.6% del total de clientes**, una proporción suficiente para realizar análisis estadísticamente significativos sin comprometer la diversidad del conjunto de datos.

2. **Nivel de Consumo**  
   Es la **ciudad con el gasto promedio más alto (≈ 39.8 mil USD)**, lo cual sugiere un perfil de cliente más activo y con mayor capacidad adquisitiva.

3. **Tasa de Membresía Premium Elevada**  
   Presenta una de las **tasas premium más altas** del conjunto, reflejando clientes con mayor nivel de fidelización hacia el negocio.

---

### Criterios Cualitativos

- Permite construir una **visión más rentable del cliente promedio**, centrada en hábitos de consumo, gasto y preferencias.  
- Facilita la **comparación futura** con otras ciudades (como Chicago o NYC) para analizar diferencias entre volumen y rentabilidad.  
- Al concentrar el análisis en una sola ciudad, se simplifica la limpieza de datos y el desarrollo de modelos exploratorios sin perder representatividad.

---

### Conclusión

El filtrado por **Miami** no solo ofrece un conjunto de datos robusto, sino que también permite focalizar los esfuerzos del análisis exploratorio en **segmentos de clientes de alto valor**.  
Este enfoque equilibra **tamaño de muestra**, **potencial de consumo** y **fidelización**, sirviendo como base ideal para los siguientes pasos del proyecto:  
- limpieza detallada
- imputación de valores faltantes  
- análisis descriptivo y visual.


In [171]:
# ============================================
# Exportación del dataset limpio del Avance #1
# ============================================

# Paso 1. Detectar la carpeta raíz del proyecto (donde está la carpeta 'data')
# Subimos un nivel desde 'notebooks' si estamos dentro
current_dir = Path.cwd()
if current_dir.name == "notebooks":
    project_dir = current_dir.parent
else:
    project_dir = current_dir

# Paso 2. Crear la ruta final hacia /data/processed/
output_dir = project_dir / "data" / "processed"
output_dir.mkdir(parents=True, exist_ok=True)

# Paso 3. Exportar el CSV del DF completo (ya limpio)
output_path = output_dir / "base_clientes_clean.csv"
df.to_csv(output_path, index=False, encoding="utf-8")

print("Archivo exportado correctamente en:")
print(output_path.resolve())

Archivo exportado correctamente en:
C:\Users\Diego Garcia\Documents\Cristian\DS-SoyHenry\Modulo 1\Código\data\processed\base_clientes_clean.csv


# Conclusiones del Avance #1 (EDA inicial, sin filtro por ciudad)

**Objetivo cumplido:** preparar una base de datos de clientes lista para las siguientes etapas del pipeline analítico, garantizando consistencia textual y estructura clara.

## Logros del avance
- **Carga y exploración** de la base de 30.000 registros y 17 variables.
- **Diagnóstico de calidad**:
  - Sin filas duplicadas.
  - Nulos relevantes en `telefono_contacto` (~50%) y `correo_electronico` (~50%).
  - Nulos moderados en `preferencias_alimenticias` (~4,68%) y bajos en `edad` (~0,34%) y `promedio_gasto_comida` (~0,48%).
- **Normalización de texto**:
  - Corrección robusta de respuestas binarias (`sí`/`no`) eliminando acentos mal decodificados y caracteres invisibles (soft hyphen, zero-width, NBSP, BOM).
  - Unificación de mayúsculas, espacios y acentos para evitar categorías “fantasma”.
- **Tabulaciones básicas** (sin filtro por ciudad):
  - `género` equilibrado (≈50/50).
  - `preferencias_alimenticias` con categorías limpias y consistentes (top: carnes, vegetariano, mariscos, vegano, pescado, otro).
  - Binarias (`consume_licor`, `ocio`, `membresia_premium`) ya **normalizadas** a `sí`/`no`.
  - `estrato_socioeconomico` apto para codificación ordinal en etapas futuras.
  - `tipo_de_pago_mas_usado` con predominio de efectivo y tarjeta; app en tercer lugar.

## Artefactos generados (para continuar)
- **Notebook de entrega:** `Avance_1_EDA.ipynb`
- **Dataset intermedio limpio (sin filtro):** `data/processed/base_clientes_clean.csv`  
  Este archivo será el **punto de partida** para las siguientes fases (limpieza avanzada, visualizaciones, modelado o integración con APIs).